In [82]:
import numpy as np
import pandas as pd
from osgeo import gdal
from gdalconst import *
#import osr
import matplotlib.pyplot as plt
%matplotlib qt




In [11]:
#Read a raster
Test_data = ("/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/")
Test = gdal.Open(Test_data + "/Mandakini.bil")#Get raster data
M_DEM = Test.ReadAsArray()

#Read stream data
name = "Mandakini_fullProfileMC_forced_0.45_3_1258909000_10_80_281_for_Arc.csv" #Set file name
path = "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/" #Set path to data
#for csv files
data = np.genfromtxt((path+name), delimiter=',', skip_header=1, names=['id', 'x', 'y', 'chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])
#for tree files
#data = np.genfromtxt((path+name), delimiter=' ', skip_header=1, names=['chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])


In [14]:



def plot_coord(Test):

    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
    dx = x_size
    dy = y_size
    nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

    xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
    yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

    #Create arrays of the x and y coordinates of each pixel (the axes)
    xcoordinates = [x*dx + xllcenter for x in range(nx)]
    ycoordinates = [y*dy + yllcenter for y in range(ny)]

    #Create 2 2d grids describing x and y coordinates
    X,Y = np.meshgrid(xcoordinates, ycoordinates) 


    fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))

    return X, Y


In [12]:
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
dx = x_size
dy = y_size
nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

#Create arrays of the x and y coordinates of each pixel (the axes)
xcoordinates = [x*dx + xllcenter for x in range(nx)]
ycoordinates = [y*dy + yllcenter for y in range(ny)]

#Create 2 2d grids describing x and y coordinates
X,Y = np.meshgrid(xcoordinates, ycoordinates) 


fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
    

In [17]:
[X, Y] = plot_coord(Test)

In [44]:
cm = plt.cm.gist_earth


fig, ax = plt.subplots(3, figsize=(10,10), sharex=False) #define 2 subfigures

ax[0].contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
ax[0].plot(data['x'], data['y'], 'ro',  markersize=2)
#ax[0].imshow(fA, interpolation='nearest', vmin=0, cmap=plt.cm.gray) 
ax[0].set_aspect('equal')
ax[0].set_title('Terrain map with stream channel')
ax[0].set_ylabel('Northing')
ax[0].set_xlabel('Easting')



ax[1].scatter(data['flow_distance'], data['elevation'], c=data['elevation'], cmap=cm)
ax[1].set_title('Stream Map')
ax[1].set_ylabel('Elevation (m)')
ax[1].set_xlabel('Distance (m)')


ax[2].scatter(data['chi'], data['elevation'], c=data['elevation'], cmap=plt.get_cmap("jet"))
ax[2].text(3, 2500, 'm/n = 0.45')
ax[2].set_ylabel('Elevation (m)')
ax[2].set_xlabel('Chi (X)')


plt.show()

In [87]:
from bokeh.mpl import to_bokeh
f = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
#p = to_bokeh(f)

In [88]:
import numpy as np
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, Label
from bokeh.layouts import layout

mn = " (m/n = 0.45)"

# output to static HTML file
output_file("linked_brushing.html")

# NEW: create a column data source for the plots to share
source = ColumnDataSource(data=dict(x0=data['flow_distance'], x1=data['chi'], y=data['elevation']))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title='Stream Profile')
left.circle('x0', 'y', source=source)
#left.title.text = "Glucose Range"
#left.xaxis.axis_label = 'Flow Distance (m)'
#left.yaxis.axis_label = 'Elevation (m)'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title='Chi Profile'+ mn)
right.circle('x1', 'y', source=source)#, legend = "m/n = 0.45")
#right.xaxis.axis_label = 'Chi (X)'
#right.yaxis.axis_label = 'Elevation (m)'

#dem = 
# put the subplots in a gridplot
p = layout([[left, right], [dem]])



# l = layout([
#   [bollinger],
#   [sliders, plot],
#   [p1, p2, p3],
# ], sizing_mode='stretch_both')


# show the results
show(p)

In [70]:
max(data['chi'])

169.116333